<a href="https://colab.research.google.com/github/StepanTita/attattr/blob/master/metrics_hate_explain_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install boto3 == 1.28.53 transformers ekphrasis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00


In [2]:
! pip install --upgrade urllib3 == 2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6


In [3]:
! git clone https: // github.com/StepanTita/attattr.git
! git clone https: // github.com/hate-alert/HateXplain.git

Cloning into 'attattr'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 56 (delta 25), reused 46 (delta 19), pack-reused 0
Receiving objects: 100% (56/56), 1.06 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Cloning into 'HateXplain'...
remote: Enumerating objects: 414, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 414 (delta 111), reused 106 (delta 99), pack-reused 280
Receiving objects: 100% (414/414), 6.76 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [4]:
import sys

sys.path.append('/content/attattr')
sys.path.append('/content/HateXplain')

In [5]:
import os
import json
import time
import datetime

import more_itertools as mit

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader
from PIL import Image, ImageDraw

from tqdm import tqdm

from transformers import BertTokenizer, BertPreTrainedModel

from sklearn.preprocessing import LabelEncoder

from HateXplain.TensorDataset.datsetSplitter import createDatasetSplit, encodeData
from HateXplain.TensorDataset.dataLoader import combine_features
from HateXplain.Preprocess.dataCollect import get_annotated_data, get_test_data
from HateXplain.Preprocess.spanMatcher import returnMask

from pytorch_pretrained_bert.model_attr import BertPooler, BertModel

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...


In [6]:
def on_gpu(f):
    def wrapper():
        if torch.cuda.is_available():
            return f()
        else:
            print('cuda unavailable')

    return wrapper

In [7]:
if torch.cuda.is_available():
    ! pip install pynvml
    from pynvml import *
    from numba import cuda


@on_gpu
def print_gpu_utilization():
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        info = nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU memory occupied: {info.used // 1024 ** 2} MB.")
    except Exception as e:
        print(e)


@on_gpu
def free_gpu_cache():
    print("Initial GPU Usage")
    print_gpu_utilization()

    torch.cuda.empty_cache()

    print("GPU Usage after emptying the cache")
    print_gpu_utilization()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00


In [8]:
print_gpu_utilization()
free_gpu_cache()

GPU memory occupied: 261 MB.
Initial GPU Usage
GPU memory occupied: 261 MB.
GPU Usage after emptying the cache
GPU memory occupied: 261 MB.


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()

In [10]:
ATTR_BATCH_SIZE = 16
ATTR_NUM_BATCH = 2

MAX_SEQ_LEN = 128

num_layer = 12
num_head = 12

In [11]:
tokenizer = BertTokenizer.from_pretrained('Hate-speech-CNERG/bert-base-uncased-hatexplain')

In [14]:
params = {
    'num_classes': 3,
    'data_file': 'HateXplain/Data/dataset.json',
    'class_label': 'HateXplain/Data/classes.npy',
    'class_names': ['hatespeech', 'normal', 'offensive'],  # ['NORMAL', 'ABUSIVE'],
    'bert_tokens': True,
    'include_special': False,

    'type_attention': 'softmax',
    'max_length': 128,
    'variance': 1,
    'decay': False,
    'majority': 2,
}

# Make more data

In [15]:
! mkdir Data
! cp HateXplain/Data/ * Data/

In [16]:
annotated_data = get_annotated_data(params)
annotated_data

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],normal
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...",4,"[Caucasian, Women]",hatespeech,2,"[Women, Caucasian]",hatespeech,3,"[Women, Caucasian]",offensive,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...",217,"[Men, Women, Other]",offensive,199,[None],offensive,215,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",offensive
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],normal
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech


In [17]:
def get_data(data):
    post_ids_list = []
    text_list = []
    attention_list = []
    label_list = []

    final_binny_output = []
    print('total_data', len(data))
    for index, row in tqdm(data.iterrows(), total=len(data)):
        annotation = row['final_label']

        text = row['text']
        post_id = row['post_id']
        annotation_list = [row['label1'], row['label2'], row['label3']]
        tokens_all = list(row['text'])

        if annotation != 'undecided':
            tokens_all, attention_masks = returnMask(row, params, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output

In [18]:
data = get_data(annotated_data)
data

total_data 20148


100%|██████████| 20148/20148 [01:17<00:00, 259.36it/s]


[['1179055004553900032_twitter',
  'normal',
  [101,
   1045,
   2123,
   2102,
   2228,
   10047,
   2893,
   2026,
   3336,
   2068,
   2317,
   2193,
   2002,
   2038,
   2048,
   2317,
   1046,
   1998,
   18368,
   2015,
   2025,
   2130,
   5028,
   102],
  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  ['normal', 'normal', 'normal']],
 ['1179063826874032128_twitter',
  'normal',
  [101,
   2057,
   3685,
   3613,
   4214,
   9731,
   10469,
   2015,
   2065,
   1996,
   2916,
   1997,
   2035,
   24185,
   22984,
   2078,
   4995,
   2102,
   8280,
   2748,
   2000,
   1037,
   4424,
   18421,
   2270,
   2862,
   2021,
   2097,
   1037,
   9099,
   11690,
   22437,
   1998,
   19483,
   24185,
   22984,
   2078,
   2022,
   2583,
   2000,
   4607,
   2037,
   2592,
   2006,
   1996,
   7316,
   7

In [19]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)
id_division

{'test': ['1178610029273976833_twitter',
  '1165785686903009283_twitter',
  '1252707503441313794_twitter',
  '1103385226921762816_twitter',
  '1169443635869487105_twitter',
  '17372469_gab',
  '1168407844095483904_twitter',
  '12643541_gab',
  '24305062_gab',
  '1123306824571158529_twitter',
  '1178857682511511552_twitter',
  '1178929320527462401_twitter',
  '9274864_gab',
  '18943330_gab',
  '1077853_gab',
  '1162547667416899586_twitter',
  '1178978611652710406_twitter',
  '1178412537496383488_twitter',
  '20989384_gab',
  '1083042797995941889_twitter',
  '1079795475560218626_twitter',
  '1108816043865042945_twitter',
  '1112504910485118979_twitter',
  '10682437_gab',
  '1178873120804999169_twitter',
  '13948927_gab',
  '1225858073270476800_twitter',
  '9551353_gab',
  '18529253_gab',
  '1187745116464832515_twitter',
  '1179048983420899328_twitter',
  '13455893_gab',
  '1100389613074558976_twitter',
  '12711894_gab',
  '1179103318762803200_twitter',
  '1179093647867949056_twitter',
  

In [20]:
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]


# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each == 1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each) == int:
            start = each
            end = each + 1
        elif len(each) == 2:
            start = each[0]
            end = each[1] + 1
        else:
            print('error')

        output.append({"docid": post_id,
                       "end_sentence": -1,
                       "end_token": end,
                       "start_sentence": -1,
                       "start_token": start,
                       "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output


# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):
    final_output = []

    if save_split:
        train_fp = open(save_path + 'train.jsonl', 'w')
        val_fp = open(save_path + 'val.jsonl', 'w')
        test_fp = open(save_path + 'test.jsonl', 'w')

    for tcount, eachrow in enumerate(dataset):
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]

        if majority_label == 'normal':
            continue

        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))

        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]

        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)

        if save_split:
            if not os.path.exists(save_path + 'docs'):
                os.makedirs(save_path + 'docs')

            with open(save_path + 'docs/' + post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))

            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp) + '\n')

            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp) + '\n')

            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp) + '\n')
            else:
                print(post_id)

    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()

    return final_output

In [21]:
! mkdir -p ./Data/Evaluation/Model_Eval

In [22]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
convert_to_eraser_format(data, method, save_split, save_path, id_division)

In [23]:
test_data = get_test_data(annotated_data, params)
test_data

Loading BERT tokenizer...


total_data 20148


100%|██████████| 20148/20148 [01:29<00:00, 224.26it/s]


,Post_id,Text,Attention,Label
0,1179055004553900032_twitter,"[101, 1045, 2123, 2102, 2228, 10047, 2893, 202...","[0.041666666666666664, 0.041666666666666664, 0...",normal
1,1179063826874032128_twitter,"[101, 2057, 3685, 3613, 4214, 9731, 10469, 201...","[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0...",normal
2,1178793830532956161_twitter,"[101, 6583, 26677, 8038, 3363, 9152, 13327, 20...","[0.09090909090909091, 0.09090909090909091, 0.0...",normal
3,1179088797964763136_twitter,"[101, 5310, 1045, 2572, 2978, 5457, 2522, 2480...","[0.029038902079856797, 0.029038902079856797, 0...",hatespeech
4,1179085312976445440_twitter,"[101, 2023, 7743, 1999, 2054, 7875, 12514, 212...","[0.02941345372281834, 0.02941345372281834, 0.0...",hatespeech
...,...,...,...,...
20143,9989999_gab,"[101, 2065, 24471, 2145, 2006, 10474, 2425, 12...","[0.018685979950755202, 0.018685979950755202, 0...",offensive
20144,9990225_gab,"[101, 2043, 1045, 2034, 2288, 2006, 2182, 1998...","[0.026476741382734967, 0.026476741382734967, 0...",offensive
20145,9991681_gab,"[101, 2001, 24532, 2102, 4315, 9587, 25016, 22...","[0.023255813953488372, 0.023255813953488372, 0...",normal
20146,9992513_gab,"[101, 2009, 2003, 9643, 2298, 2012, 2088, 2832...","[0.03505130729982109, 0.03505130729982109, 0.0...",hatespeech


In [24]:
def transform_data(data, max_len):
    def pad_truncate(record):
        out = record
        if len(out) > max_len:
            out = out[:max_len]
        elif len(out) < max_len:
            out = out + [0] * (max_len - len(out))

        return out

    input_ids = []
    segment_ids = []
    # rationale = []
    attention_mask = []
    labels = []
    for e in tqdm(data):
        attention_mask.append(np.array([1] * len(e[1]) + [0] * (max_len - len(e[0]))))
        input_ids.append(np.array(pad_truncate(e[0])))
        segment_ids.append(np.array([0] * max_len))
        # rationale.append(np.array(pad_truncate(e[1])))
        if e[2] == 'hatespeech':
            labels.append(0)
        elif e[2] == 'normal':
            labels.append(1)
        else:
            labels.append(2)
    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long),
        'token_type_ids': torch.tensor(segment_ids, dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        # 'rationale': torch.tensor(rationale, dtype=torch.float),
        'label_ids': torch.tensor(labels, dtype=torch.long),
    }

In [25]:
val_data = test_data[test_data['Post_id'].isin(id_division['val'])]
test_data = test_data[test_data['Post_id'].isin(id_division['test'])]

In [26]:
test_final = transform_data(encodeData(test_data, None, params), MAX_SEQ_LEN)
test_final

100%|██████████| 1924/1924 [00:00<00:00, 30698.05it/s]
<ipython-input-24-ecc3cae14b6a>:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  'input_ids': torch.tensor(input_ids, dtype=torch.long),


{'input_ids': tensor([[ 101, 1045, 2123,  ...,    0,    0,    0],
         [ 101, 6874, 8840,  ...,    0,    0,    0],
         [ 101, 5310, 2054,  ...,    0,    0,    0],
         ...,
         [ 101, 2129, 2003,  ...,    0,    0,    0],
         [ 101, 2092, 2026,  ...,    0,    0,    0],
         [ 101, 2021, 2009,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'label_ids': tensor([1, 0, 1,  ..., 2, 0, 1])}

In [27]:
def scaled_input(emb, attr_batch_size, attr_num_batch, baseline=None, start_i=None, end_i=None):
    # shape of emb: (num_head, seq_len, seq_len)
    if baseline is None:
        baseline = torch.zeros_like(emb)

    num_points = attr_batch_size * attr_num_batch
    scale = 1.0 / num_points
    if start_i is None:
        step = (emb.unsqueeze(0) - baseline.unsqueeze(0)) * scale  # (1, num_head, seq_len, seq_len)
        # n = batch_size * num_batch
        res = torch.cat([torch.add(baseline.unsqueeze(0), step * i) for i in range(num_points)], dim=0)  # (n, num_head, seq_len, seq_len)
        return res, step[0]
    else:
        step = (emb - baseline) * scale
        start_emb = torch.add(baseline, step * start_i)
        end_emb = torch.add(baseline, step * end_i)
        step_new = (end_emb.unsqueeze(0) - start_emb.unsqueeze(0)) * scale
        res = torch.cat([torch.add(start_emb.unsqueeze(0), step_new * i) for i in range(num_points)], dim=0)
        return res, step_new[0]

In [61]:
def generate_attr_score(model, input_ids, segment_ids, attention_mask, label_ids, attr_batch_size, attr_num_batch):
    res_attr = []

    layers_attr = []

    for tar_layer in range(num_layer):
        att, baseline_logits, _ = model(input_ids=input_ids, token_type_ids=segment_ids,
                                        attention_mask=attention_mask,
                                        labels=label_ids, tar_layer=tar_layer)
        pred_label = torch.argmax(baseline_logits, dim=1)

        scale_att, step = scaled_input(att.data, attr_batch_size, attr_num_batch)
        scale_att.requires_grad_(True)

        attr_all = None
        for j_batch in range(attr_num_batch):
            one_batch_att = scale_att[j_batch * attr_batch_size:(j_batch + 1) * attr_batch_size]
            tar_prob, grad = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=attention_mask,
                                   labels=label_ids, tar_layer=tar_layer, tmp_score=one_batch_att,
                                   pred_label=pred_label)
            grad = grad.sum(dim=0)
            attr_all = grad if attr_all is None else torch.add(attr_all, grad)
        layers_attr.append(attr_all.data.cpu().detach().numpy())

    return np.array(layers_attr).sum(1).sum(1)

In [29]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [30]:
def return_dataloader(input_ids, token_type_ids, att_masks, labels, batch_size):
    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels, dtype=torch.long)
    masks = torch.tensor(np.array(att_masks), dtype=torch.uint8)
    data = TensorDataset(inputs, token_type_ids, masks, labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

In [31]:
def convert_data(test_data, params, list_dict, rational_present=True, topk=2):
    """this converts the data to be with or without the rationals based on the previous predictions"""
    """input: params -- input dict, list_dict -- previous predictions containing rationals
    rational_present -- whether to keep rational only or remove them only
    topk -- how many words to select"""

    temp_dict = {}
    for ele in list_dict:
        temp_dict[ele['annotation_id']] = ele['rationales'][0]['soft_rationale_predictions']

    test_data_modified = []

    for index, row in tqdm(test_data.iterrows(), total=len(test_data)):
        try:
            attention = temp_dict[row['Post_id']]
        except KeyError:
            continue
        topk_indices = sorted(range(len(attention)), key=lambda i: attention[i])[-topk:]
        new_text = []
        new_attention = []
        if rational_present:
            new_attention = [0]
            new_text = [101]
            for i in range(len(row['Text'])):
                if (i in topk_indices):
                    new_text.append(row['Text'][i])
                    new_attention.append(row['Attention'][i])
            new_attention.append(0)
            new_text.append(102)
        else:
            for i in range(len(row['Text'])):
                if i not in topk_indices:
                    new_text.append(row['Text'][i])
                    new_attention.append(row['Attention'][i])
        test_data_modified.append([row['Post_id'], new_text, new_attention, row['Label']])

    df = pd.DataFrame(test_data_modified, columns=test_data.columns)
    return df

In [55]:
def standaloneEval_with_rational(model, test_data, attr_batch_size, attr_num_batch, max_len, topk):
    test_ready = transform_data(encodeData(test_data, None, params), max_len)

    post_ids = test_data['Post_id'].tolist()

    test_dataloader = return_dataloader(
        test_ready['input_ids'],
        test_ready['token_type_ids'],
        test_ready['attention_mask'],
        test_ready['label_ids'],
        1
    )

    model.eval()

    print("Running eval on test data...")
    t0 = time.time()

    true_labels = []
    pred_labels = []
    logits_all = []
    attention_all = []
    input_mask_all = []

    # Evaluate data for one epoch
    for step, (input_ids, segment_ids, attention_mask, label_ids) in tqdm(enumerate(test_dataloader),
                                                                          total=len(test_dataloader)):

        # Progress update every 5 batches.
        if step % 5 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            print('Elapsed: ', elapsed)

        input_ids = input_ids.to(device)
        segment_ids = segment_ids.to(device)
        attention_mask = attention_mask.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            _, logits, token_logits = model(input_ids=input_ids, token_type_ids=segment_ids,
                                            attention_mask=attention_mask, tar_layer=-1)
            logits = torch.nn.functional.softmax(logits, dim=-1)

        attr_score = generate_attr_score(model, input_ids, segment_ids, attention_mask, label_ids, attr_batch_size,
                                         attr_num_batch)

        # (num_layer, batch_size, num_heads, sequence_length, sequence_length) ->
        # (batch_size, num_heads, sequence_length) -> (batch_size, sequence_length)
        # attention_vectors=np.mean(outputs[1][11][:,:,0,:].detach().cpu().numpy(),axis=1)

        # attr_score should be: (batch_size, sequence_length)

        pred_labels += list(np.argmax(logits.detach().cpu().numpy(), axis=1).flatten())
        true_labels += list(label_ids.detach().cpu().numpy().flatten())
        attention_all += list(attr_score[0])
        input_mask_all += list(attention_mask.detach().cpu().numpy())
        logits_all += list(logits)

    attention_vector_final = []
    for x, y in zip(attention_all, input_mask_all):
        temp = []
        for x_ele, y_ele in zip(x, y):
            if y_ele == 1:
                temp.append(x_ele)
        attention_vector_final.append(temp)

    list_dict = []

    for post_id, attention, logits, pred, ground_truth in zip(post_ids, attention_vector_final, logits_all, pred_labels,
                                                              true_labels):
        temp = {}
        encoder = LabelEncoder()
        encoder.classes_ = np.load('./Data/classes.npy')
        pred_label = encoder.inverse_transform([pred])[0]
        ground_label = encoder.inverse_transform([ground_truth])[0]

        temp['annotation_id'] = post_id
        temp['classification'] = pred_label
        temp['classification_scores'] = {'hatespeech': logits[0].item(), 'normal': logits[1].item(),
                                         'offensive': logits[2].item()}

        topk_indicies = sorted(range(len(attention)), key=lambda i: attention[i])[-topk:]

        temp_hard_rationales = []
        for ind in topk_indicies:
            temp_hard_rationales.append({'end_token': ind + 1, 'start_token': ind})

        temp['rationales'] = [
            {
                'docid': post_id,
                'hard_rationale_predictions': temp_hard_rationales,
                'soft_rationale_predictions': attention,
                #"soft_sentence_predictions":[1.0],
                'truth': ground_truth
            }
        ]
        list_dict.append(temp)

    return list_dict

In [56]:
def get_final_dict_with_rational(model, test_data, params, attr_batch_size, attr_num_batch, max_len, topk=5):
    list_dict_org = standaloneEval_with_rational(model, test_data, attr_batch_size, attr_num_batch, max_len, topk=topk)
    test_data_with_rational = convert_data(test_data, params, list_dict_org, rational_present=True, topk=topk)

    list_dict_with_rational = standaloneEval_with_rational(model, test_data_with_rational, attr_batch_size,
                                                           attr_num_batch, max_len, topk=topk)
    test_data_without_rational = convert_data(test_data, params, list_dict_org, rational_present=False, topk=topk)

    list_dict_without_rational = standaloneEval_with_rational(model, test_data_without_rational, attr_batch_size,
                                                              attr_num_batch,
                                                              max_len, topk=topk)

    final_list_dict = []
    for ele1, ele2, ele3 in zip(list_dict_org, list_dict_with_rational, list_dict_without_rational):
        ele1['sufficiency_classification_scores'] = ele2['classification_scores']
        ele1['comprehensiveness_classification_scores'] = ele3['classification_scores']
        final_list_dict.append(ele1)
    return final_list_dict

In [58]:
class SC_weighted_BERT(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config, add_pooling_layer=True)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                attn=None,
                labels=None,
                tar_layer=None,
                tmp_score=None,
                pred_label=None):

        _, sequence_output, att_score = self.bert(
            input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,
            output_all_encoded_layers=False,
            tar_layer=tar_layer, tmp_score=tmp_score)

        pooled_output = sequence_output

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        prob = torch.nn.functional.softmax(logits)

        if labels is not None:
            tar_prob = prob[:, labels[0]]

        if tmp_score is None:
            return att_score[0], logits, None
        else:
            gradient = torch.autograd.grad(torch.unbind(torch.gather(prob, dim=1, index=pred_label.view(-1, 1))),
                                           tmp_score)
            return tar_prob, gradient[0]

In [59]:
model = SC_weighted_BERT.from_pretrained('Hate-speech-CNERG/bert-base-uncased-hatexplain').to(device)
model

SC_weighted_BERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
         

In [62]:
dict_rational = get_final_dict_with_rational(model, test_data[test_data['Label'] != 'normal'][:32], params,
                                             attr_batch_size=ATTR_BATCH_SIZE, attr_num_batch=ATTR_NUM_BATCH,
                                             max_len=MAX_SEQ_LEN,
                                             topk=5)
dict_rational

100%|██████████| 32/32 [00:00<00:00, 8323.58it/s]
<ipython-input-30-320dbf853288>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(input_ids)
<ipython-input-30-320dbf853288>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Running eval on test data...


  0%|          | 0/32 [00:00<?, ?it/s]<ipython-input-58-fbd97c2e2750>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = torch.nn.functional.softmax(logits)
  0%|          | 0/32 [00:00<?, ?it/s]


TypeError: ignored

In [43]:
! mkdir explanations_dicts

In [44]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """

    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):  #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [46]:
# for item in dict_rational:
#     for key in ['classification_scores', 'sufficiency_classification_scores', 'comprehensiveness_classification_scores']:
#         for k, v in item[key].items():
#             item[key][k] = v.item()

In [47]:
path_name_explanation = 'explanations_dicts/bert_top5.json'
with open(path_name_explanation, 'w') as fp:
    fp.write('\n'.join(json.dumps(i, cls=NumpyEncoder) for i in dict_rational))

In [51]:
with open('Data/Evaluation/Model_Eval/test.jsonl', 'r') as fp, open('Data/Evaluation/Model_Eval/test1.jsonl',
                                                                    'w') as out:
    for line in fp.readlines():
        if json.loads(line)['annotation_id'] in list(test_data[test_data['Label'] != 'normal'][:32]['Post_id']):
            out.write(line)

In [53]:
! cd HateXplain/eraserbenchmark/ & & PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir../../Data/Evaluation/Model_Eval --results../../explanations_dicts/bert_top5.json --score_file../../model_explain_output.json

  1745 MainThread Error in instances: 0 instances fail validation: set()
  1814 MainThread No sentence level predictions detected, skipping sentence-level diagnostic
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio

In [54]:
with open('model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])


Plausibility
IOU F1 : 0.12709497206703912
Token F1 : 0.2553957575938176
AUPRC : 0.3460637209474556

Faithfulness
Comprehensiveness : -0.010001594200730324
Sufficiency 0.0
